In [32]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [33]:
from typing import *
import numpy as np
from functools import partial
from fastprogress import progress_bar
import pandas as pd
import h5py

from lumin.plotting.results import plot_roc

import torch
from torch import Tensor, nn
import torch.nn.functional as F
from torch._vmap_internals import _vmap as vmap

from tomopt.volume import *
from tomopt.muon import *
from tomopt.inference import *
from tomopt.optimisation import *
from tomopt.core import *
from tomopt.utils import *
from tomopt.plotting import *

import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
DEVICE = torch.device("cpu")

In [35]:
def area_cost(x:Tensor) -> Tensor:
    return F.relu(x)

In [36]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.9
    init_res = 1e4
    n_panels = 4
    layers.append(PanelDetectorLayer(pos='above', lw=lwh[:2], z=1, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.5,0.5,1-(i*(2*size)/n_panels)], init_xy_span=[2.,2.],
                                                      area_cost_func=area_cost, device=DEVICE) for i in range(n_panels)]))
    for z in [0.8,0.7,0.6,0.5,0.4,0.3]:
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=DEVICE))
    layers.append(PanelDetectorLayer(pos='below', lw=lwh[:2], z=0.2, size=2*size,
                                     panels=[DetectorPanel(res=init_res, eff=init_eff,
                                                      init_xyz=[0.5,0.5,0.2-(i*(2*size)/n_panels)], init_xy_span=[2.,2.],
                                                      area_cost_func=area_cost, device=DEVICE) for i in range(n_panels)]))

    return nn.ModuleList(layers) 

In [37]:
volume = Volume(get_layers())

In [38]:
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([1.]), and xy span tensor([2., 2.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.9500]), and xy span tensor([2., 2.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.9000]), and xy span tensor([2., 2.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.8500]), and xy span tensor([2., 2.])
      )
    )
    (1): PassiveLayer()
    (2): PassiveLayer()
    (3): PassiveLayer()
    (4): PassiveLayer()
    (5): PassiveLayer()
    (6): PassiveLayer()
    (7): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.2000]),

In [39]:
from tomopt.optimisation import ULorryPassiveGenerator

In [40]:
u_volume = 12
passive_gen = ULorryPassiveGenerator(volume, u_volume=u_volume, u_prob=0.5, fill_frac=0.8, x0_lorry=X0['iron'], bkg_materials=['air', 'iron'])

In [41]:
test_passives = PassiveYielder(passive_gen, n_passives=200)

In [42]:
from tomopt.optimisation import NoMoreNaNs, PanelMetricLogger, CostCoefWarmup, PanelOptConfig, MuonResampler

In [43]:
wrapper = PanelVolumeWrapper(volume,
                             xy_pos_opt=partial(torch.optim.SGD, lr=5e4),
                             z_pos_opt=partial(torch.optim.SGD, lr=5e3),
                             xy_span_opt=partial(torch.optim.SGD, lr=1e4),
                             loss_func=VolumeClassLoss(x02id={0:0, 1:1}, target_budget=5),
                             partial_volume_inferer=partial(DenseBlockClassifierFromX0s, n_block_voxels=u_volume, partial_x0_inferer=PanelX0Inferer, volume=volume, ratio_offset=-1, ratio_coef=1))

In [44]:
def set_detectors(volume) -> None:  
    for i,p in enumerate(volume.get_detectors()[0].panels):
        p.z.data = 1-(torch.rand(1)*0.2)

    for i,p in enumerate(volume.get_detectors()[1].panels):
        p.z.data = 0.2-(torch.rand(1)*0.2)

In [45]:
set_detectors(volume)

In [46]:
volume

Volume(
  (layers): ModuleList(
    (0): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.8587]), and xy span tensor([2., 2.])
        (1): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.8190]), and xy span tensor([2., 2.])
        (2): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.8347]), and xy span tensor([2., 2.])
        (3): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.8592]), and xy span tensor([2., 2.])
      )
    )
    (1): PassiveLayer()
    (2): PassiveLayer()
    (3): PassiveLayer()
    (4): PassiveLayer()
    (5): PassiveLayer()
    (6): PassiveLayer()
    (7): PanelDetectorLayer(
      (panels): ModuleList(
        (0): <class 'tomopt.volume.panel.DetectorPanel'> located at xy=tensor([0.5000, 0.5000]), z=tensor([0.057

In [47]:
preds = wrapper.predict(test_passives,
                n_mu_per_volume=250,
                mu_bs=250,
                pred_cb=ClassPredHandler(x02id={0:0, 1:1}),
                cbs=[MuonResampler()])

/home/centos/volume/mode_muon_tomography/tomopt/inference/scattering.py:329: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  idxs = torch.combinations(torch.arange(0, unc.shape[-1]), with_replacement=True)


In [96]:
from sklearn.metrics import roc_auc_score

def get_roc_auc(preds:List[Tuple[np.ndarray,np.ndarray]], n=1000) -> Tuple[float, float]:
    scores = []
    preds = np.array(preds, dtype=object).flatten().reshape(-1,2)
    for _ in range(n):
        rpreds = preds[np.random.choice(np.arange(len(preds)), len(preds), replace=True)]
        if len(np.unique(rpreds[:,1])) == 2:  # Avoid single class in sampling
            scores.append(roc_auc_score(y_true=rpreds[:,1].astype(int), y_score=rpreds[:,0]))
    return np.mean(scores), np.std(scores, ddof=1)

In [99]:
get_roc_auc(preds)

(0.9019387403059502, 0.023147841100060174)

In [104]:
def get_det_data(volume:Volume) -> np.ndarray:
    z = []
    for d in volume.get_detectors():
        for p in d.panels:
            z.append(p.z.detach().cpu().item())
    z = sorted(z)[::-1]
    return np.array(z)

In [105]:
get_det_data(volume)

array([0.85917819, 0.85871094, 0.83474422, 0.81897539, 0.17303939,
       0.1658545 , 0.13859355, 0.05764951])

In [ ]:
df = pd.DataFrame()
for _ in progress_bar(range(450)):
    set_detectors(volume)
    preds = wrapper.predict(test_passives,
                n_mu_per_volume=250,
                mu_bs=250,
                pred_cb=ClassPredHandler(x02id={0:0, 1:1}),
                cbs=[MuonResampler()])
    df = df.append([(*get_det_data(volume), *get_roc_auc(preds))])
    df.to_csv('z_sep_data.csv')

In [140]:
df

,0,1,2,3,4,5,6,7,8,9
0,0.986205,0.976234,0.923943,0.911440,0.168576,0.145932,0.133829,0.083904,NaN,NaN
0,0.975964,0.951431,0.899263,0.840796,0.184532,0.051205,0.050029,0.016397,1.0,0.0


In [136]:
df.append([(*get_det_data(volume), *get_roc_auc(preds))])

/home/centos/volume/anaconda3/envs/tomopt/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/centos/volume/anaconda3/envs/tomopt/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/centos/volume/anaconda3/envs/tomopt/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/centos/volume/anaconda3/envs/tomopt/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/centos/volume/anaconda3/envs/tomopt/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type

,0,1,2,3,4,5,6,7,8,9
0,0.989391,0.976199,0.937046,0.896724,0.153803,0.125483,0.092282,0.0229,NaN,NaN


(0.9936212301254272,
 0.9741640686988831,
 0.8986803293228149,
 0.8865054249763489,
 0.12273698300123215,
 0.11773987859487534,
 0.05452634394168854,
 0.03074696660041809,
 1.0,
 0.0)

In [127]:
df.append(data).append(data)

,0,1,2
0,"[0.9979848265647888, 0.9253425598144531, 0.817...",1.0,0.0
1,"[0.9503454566001892, 0.9196286797523499, 0.884...",1.0,0.0
0,"[0.9979848265647888, 0.9253425598144531, 0.817...",1.0,0.0
1,"[0.9503454566001892, 0.9196286797523499, 0.884...",1.0,0.0


In [118]:
df

,z_pos,roc_auc,roc_auc_unc
